### LSTM Model

In [0]:
# restart kernel after running this cell
# !pip install pandas --upgrade

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.24.2)


In [1]:
import pandas as pd
import numpy as np 
import scipy
import re
import ast
import pickle
import time
import copy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import networkx as nx
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import cdist, cosine
import time
import multiprocessing
from gensim.models import Word2Vec, FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from datetime import datetime
import os
import glob
import re
import math
import json
import random

from IPython import display
from matplotlib import cm

import tensorflow as tf
from tensorflow.python.data import Dataset

# from google.colab import drive
# drive.mount('/content/drive')

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from itertools import chain
import time
import multiprocessing
import copy
import pickle
import datetime as dt
import h5py
import re
from subprocess import call
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.models import load_model
from keras import Model
from keras import backend as K
from keras.utils import to_categorical
import gc
from keras.layers import Input, LSTM, Bidirectional, Dense, Embedding, Flatten
from gensim.models import Word2Vec
%matplotlib inline

import nltk
nltk.download('punkt')
nltk.download('stopwords')

C:\Users\Wei Han\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to C:\Users\Wei
[nltk_data]     Han\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Wei
[nltk_data]     Han\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
combined_p = pd.read_pickle("./Data/combined.p")

In [9]:
combined_p.columns.tolist()

['time',
 'assetCode',
 'assetName',
 'volume',
 'close',
 'open',
 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1',
 'returnsClosePrevMktres1',
 'returnsOpenPrevMktres1',
 'returnsClosePrevRaw10',
 'returnsOpenPrevRaw10',
 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10',
 'returnsOpenNextMktres10',
 'universe',
 'open_close',
 'oc_average',
 'turnover',
 'open_close_relative',
 'turnover_relative',
 'volume_relative',
 'returnsOpenPrevMktres1_relative',
 'returnsOpenPrevMktres10_relative',
 'urgency_min',
 'urgency_count',
 'takeSequence_min',
 'takeSequence_max',
 'bodySize_mean',
 'wordCount_mean',
 'sentenceCount_mean',
 'companyCount_mean',
 'marketCommentary_mean',
 'relevance_mean',
 'sentimentNegative_mean',
 'sentimentNeutral_mean',
 'sentimentPositive_mean',
 'sentimentWordCount_mean',
 'noveltyCount12H_mean',
 'noveltyCount24H_mean',
 'noveltyCount3D_mean',
 'noveltyCount5D_mean',
 'noveltyCount7D_mean',
 'volumeCounts12H_mean',
 'volumeCounts24H_mean',
 'volumeCounts3

In [2]:
%%time
combined_data = pd.read_pickle('./Data/new_df.p')

Wall time: 9.71 s


In [3]:
combined_data.sort_values('time', inplace=True)

In [2]:
# store resuls
results = {}

In [5]:
%%time

# month from 0 to 11
combined_data['month'] = combined_data['month'] - 1

# missing value imputation

group_mean_features = [
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
]

combined_data['headline'].fillna('', inplace=True)
combined_data['urgency_min'].fillna(5.0, inplace=True)

combined_data[group_mean_features] = combined_data.groupby("assetCode")[group_mean_features].transform(lambda x: x.fillna(x.mean()))

combined_data = combined_data[~combined_data['returnsClosePrevMktres10'].isnull()]

combined_data.fillna(0, inplace=True)

Wall time: 16 s


In [6]:
combined_data.shape

(3916997, 101)

In [9]:
combined_data.columns

Index(['time', 'assetCode', 'assetName', 'volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       ...
       'noveltyCount3D_mean_stan', 'noveltyCount5D_mean_stan',
       'noveltyCount7D_mean_stan', 'volumeCounts12H_mean_stan',
       'volumeCounts24H_mean_stan', 'volumeCounts3D_mean_stan',
       'volumeCounts5D_mean_stan', 'volumeCounts7D_mean_stan',
       'dayofweek_stan', 'month_stan'],
      dtype='object', length=101)

In [10]:
combined_data.tail()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,...,noveltyCount3D_mean_stan,noveltyCount5D_mean_stan,noveltyCount7D_mean_stan,volumeCounts12H_mean_stan,volumeCounts24H_mean_stan,volumeCounts3D_mean_stan,volumeCounts5D_mean_stan,volumeCounts7D_mean_stan,dayofweek_stan,month_stan
4053824,2016-12-30,WEB.O,Web.com Group Inc,313808.0,21.150000,21.049999,0.007143,0.009592,0.017020,0.008069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2735347,2016-12-30,FMBI.O,First Midwest Bancorp Inc,241696.0,25.230000,25.230000,-0.000792,-0.002372,0.000730,-0.002542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1554928,2016-12-30,NG.A,NovaGold Resources Inc,2227458.0,4.560000,4.740000,-0.031847,0.062780,-0.029414,0.063522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4070552,2016-12-30,PSXP.N,Phillips 66 Partners LP,241176.0,48.639999,48.000000,0.012490,-0.012346,0.015263,-0.013469,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2967626,2016-12-30,GTLS.O,Chart Industries Inc,183875.0,36.020000,36.400002,-0.011526,-0.003286,-0.002773,-0.005509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
combined_data['assetCode'].unique().shape

(3430,)

In [0]:
# %%time

# # sampling

# stocks = train_data['assetCode'].unique()

# # N = min(100, stocks.shape[0])
# N = stocks.shape[0]

# np.random.seed(1)
# selected_stocks = set(np.random.choice(stocks, N))
# train_data = train_data[train_data['assetCode'].map(lambda x: x in selected_stocks)]
# test_data = test_data[test_data['assetCode'].map(lambda x: x in selected_stocks)]

### Model Building v1

In [6]:
%%time

v_str = 'v1'

train_end_years = [2011, 2012, 2013, 2014, 2015, 2016]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 88s 244us/step - loss: 0.6682 - acc: 0.5884 - val_loss: 0.7136 - val_acc: 0.5301
current sigma: 0.263989
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 100s 278us/step - loss: 0.6473 - acc: 0.6185 - val_loss: 0.7216 - val_acc: 0.5315
current sigma: 0.291399
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 94s 260us/step - loss: 0.6319 - acc: 0.6358 - val_loss: 0.7355 - val_acc: 0.5262
current sigma: 0.240066
Year 2011: sigma=0.240066
[[46519 41597]
 [36974 40726]]
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==============================] - 86s 234us/step - loss: 0.6679 - acc: 0.5930 - val_loss: 0.6866 - val_acc: 0.5653
current sigma: 0.825409
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [=

In [7]:
filename = "sliding_final_output_from_results_dict_V1.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### Model Building v2

In [7]:
%%time

v_str = 'v2'

train_end_years = [2011, 2012, 2013, 2014, 2015, 2016]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 86s 237us/step - loss: 0.6691 - acc: 0.5866 - val_loss: 0.7100 - val_acc: 0.5332
current sigma: 0.265445
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 87s 241us/step - loss: 0.6499 - acc: 0.6148 - val_loss: 0.7174 - val_acc: 0.5300
current sigma: 0.271875
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 88s 243us/step - loss: 0.6349 - acc: 0.6338 - val_loss: 0.7339 - val_acc: 0.5214
current sigma: 0.238026
Year 2011: sigma=0.238026
[[45092 43024]
 [36329 41371]]
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==============================] - 87s 234us/step - loss: 0.6681 - acc: 0.5919 - val_loss: 0.6868 - val_acc: 0.5641
current sigma: 0.827156
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==

In [8]:
filename = "sliding_final_output_from_results_dict_V1_and_V2.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### Model Building v4

In [6]:
%%time

# remark for Wei Han: add industry-neutralised sentiments

v_str = 'v4'

train_end_years = [2011, 2012, 2013, 2014, 2015, 2016]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'sentimentNegative_mean_stan',
    'sentimentNeutral_mean_stan',
    'sentimentPositive_mean_stan',
    'sentimentWordCount_mean_stan',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 93s 257us/step - loss: 0.6697 - acc: 0.5858 - val_loss: 0.7101 - val_acc: 0.5344
current sigma: 0.264845
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 85s 237us/step - loss: 0.6505 - acc: 0.6150 - val_loss: 0.7170 - val_acc: 0.5273
current sigma: 0.269737
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 84s 234us/step - loss: 0.6353 - acc: 0.6334 - val_loss: 0.7310 - val_acc: 0.5212
current sigma: 0.251577
Year 2011: sigma=0.251577
[[43324 44792]
 [34606 43094]]
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==============================] - 84s 228us/step - loss: 0.6691 - acc: 0.5909 - val_loss: 0.6870 - val_acc: 0.5642
current sigma: 0.811922
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==

In [7]:
filename = "sliding_final_output_from_results_dict_V4.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### Model Building v5

In [6]:
%%time

# remark for Wei Han: add other industry-neutralised features

v_str = 'v5'

train_end_years = [2011, 2012, 2013, 2014, 2015, 2016]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
    'returnsClosePrevRaw1_stan',
    'returnsOpenPrevRaw1_stan',
    'returnsClosePrevMktres1_stan',
    'returnsOpenPrevMktres1_stan',
    'returnsClosePrevRaw10_stan',
    'returnsOpenPrevRaw10_stan',
    'returnsClosePrevMktres10_stan',
    'returnsOpenPrevMktres10_stan',
    'open_close_stan',
    'oc_average_stan',
    'turnover_stan',
    'open_close_relative_stan',
    'turnover_relative_stan',
    'volume_relative_stan',
    'returnsOpenPrevMktres1_relative_stan',
    'returnsOpenPrevMktres10_relative_stan',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'sentimentNegative_mean_stan',
    'sentimentNeutral_mean_stan',
    'sentimentPositive_mean_stan',
    'sentimentWordCount_mean_stan',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 94s 259us/step - loss: 0.6696 - acc: 0.5857 - val_loss: 0.7175 - val_acc: 0.5237
current sigma: 0.173767
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 97s 269us/step - loss: 0.6477 - acc: 0.6174 - val_loss: 0.7339 - val_acc: 0.5217
current sigma: 0.216586
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 96s 266us/step - loss: 0.6319 - acc: 0.6359 - val_loss: 0.7479 - val_acc: 0.5187
current sigma: 0.208302
Year 2011: sigma=0.208302
[[38651 49465]
 [30346 47354]]
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==============================] - 90s 243us/step - loss: 0.6690 - acc: 0.5909 - val_loss: 0.6883 - val_acc: 0.5626
current sigma: 0.792931
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==

In [7]:
filename = "sliding_final_output_from_results_dict_V5.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### Model Building v6

In [6]:
%%time

# remark for Wei Han: add sentiments and other industry-neutralised features from v5

v_str = 'v6'

train_end_years = [2011, 2012, 2013, 2014, 2015, 2016]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
    'returnsClosePrevRaw1_stan',
    'returnsOpenPrevRaw1_stan',
    'returnsClosePrevMktres1_stan',
    'returnsOpenPrevMktres1_stan',
    'returnsClosePrevRaw10_stan',
    'returnsOpenPrevRaw10_stan',
    'returnsClosePrevMktres10_stan',
    'returnsOpenPrevMktres10_stan',
    'open_close_stan',
    'oc_average_stan',
    'turnover_stan',
    'open_close_relative_stan',
    'turnover_relative_stan',
    'volume_relative_stan',
    'returnsOpenPrevMktres1_relative_stan',
    'returnsOpenPrevMktres10_relative_stan',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'sentimentNegative_mean_stan',
    'sentimentNeutral_mean_stan',
    'sentimentPositive_mean_stan',
    'sentimentWordCount_mean_stan',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)
    x = Dropout(.2)(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 77s 214us/step - loss: 0.6733 - acc: 0.5808 - val_loss: 0.7168 - val_acc: 0.5303
current sigma: 0.171958
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 86s 239us/step - loss: 0.6530 - acc: 0.6098 - val_loss: 0.7224 - val_acc: 0.5323
current sigma: 0.266378
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 91s 253us/step - loss: 0.6389 - acc: 0.6282 - val_loss: 0.7293 - val_acc: 0.5322
current sigma: 0.296818
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 106s 295us/step - loss: 0.6286 - acc: 0.6402 - val_loss: 0.7391 - val_acc: 0.5291
current sigma: 0.301103
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 94s 261us/step - loss: 0

In [7]:
filename = "sliding_final_output_from_results_dict_V6.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### Model Building v7

In [6]:
%%time

# remark for Wei Han: add sentiments and other industry-neutralised features from v5

v_str = 'v7'

train_end_years = [2011, 2012, 2013, 2014, 2015, 2016]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
    'returnsClosePrevRaw1_stan',
    'returnsOpenPrevRaw1_stan',
    'returnsClosePrevMktres1_stan',
    'returnsOpenPrevMktres1_stan',
    'returnsClosePrevRaw10_stan',
    'returnsOpenPrevRaw10_stan',
    'returnsClosePrevMktres10_stan',
    'returnsOpenPrevMktres10_stan',
    'open_close_stan',
    'oc_average_stan',
    'turnover_stan',
    'open_close_relative_stan',
    'turnover_relative_stan',
    'volume_relative_stan',
    'returnsOpenPrevMktres1_relative_stan',
    'returnsOpenPrevMktres10_relative_stan',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'sentimentNegative_mean_stan',
    'sentimentNeutral_mean_stan',
    'sentimentPositive_mean_stan',
    'sentimentWordCount_mean_stan',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(.2)(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 75s 209us/step - loss: 0.6737 - acc: 0.5781 - val_loss: 0.7116 - val_acc: 0.5323
current sigma: 0.212841
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 82s 226us/step - loss: 0.6551 - acc: 0.6079 - val_loss: 0.7214 - val_acc: 0.5262
current sigma: 0.254792
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 87s 241us/step - loss: 0.6399 - acc: 0.6273 - val_loss: 0.7325 - val_acc: 0.5223
current sigma: 0.216180
Year 2011: sigma=0.216180
[[41366 46750]
 [32460 45240]]
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==============================] - 86s 233us/step - loss: 0.6734 - acc: 0.5833 - val_loss: 0.6862 - val_acc: 0.5631
current sigma: 0.743157
Train on 369432 samples, validate on 168336 samples
Epoch 1/1
369432/369432 [==

In [7]:
filename = "sliding_final_output_from_results_dict_V7.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

In [29]:
filename = "final_output_from_results_dict.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### Model Building v3

In [6]:
original_combined_df = pd.read_pickle('./Data/combined.p')

###############################################
### RESHAPING COMBINED_DF ACCORDING TO TIME ###
# month from 0 to 11
original_combined_df['month'] = original_combined_df['month'] - 1

# missing value imputation

group_mean_features = [
  'returnsClosePrevMktres1',
  'returnsOpenPrevMktres1',
  'returnsClosePrevMktres10',
  'returnsOpenPrevMktres10',
  'returnsOpenPrevMktres1_relative',
  'returnsOpenPrevMktres10_relative',
]

original_combined_df['headline'].fillna('', inplace=True)
original_combined_df['urgency_min'].fillna(5.0, inplace=True)

original_combined_df[group_mean_features] = original_combined_df.groupby("assetCode")[group_mean_features].transform(lambda x: x.fillna(x.mean()))

original_combined_df = original_combined_df[~original_combined_df['returnsClosePrevMktres10'].isnull()]

original_combined_df.fillna(0, inplace=True) ### <--- up to this point to get original df in same state

In [10]:
industry_df = pd.read_csv('./Data/IndustryData.csv')

In [11]:
industry_df.columns.tolist()

['assetCode', 'Description', 'Employees', 'Industry']

In [8]:
def shape_wv(word2vec_features, train_start, train_end, combined_df, industry_df):

  combined_df = combined_df.set_index('time')[train_start: train_end]
  print(combined_df.shape)
  ### RESHAPING COMBINED_DF ACCORDING TO TIME ###
  ###############################################

  assetCode = combined_df['assetCode'].values

  print("word2vec:", len(word2vec_features))

  word2vec_features['assetCode'] = assetCode
  
  word2vec_features = pd.merge(word2vec_features, industry_df, how='inner', on='assetCode')
  word2vec_features = word2vec_features[pd.notnull(word2vec_features['Industry'])]
  word2vec_features = word2vec_features.drop(['Industry'], axis=1)
  
  return word2vec_features

In [9]:
%%time

v_str = 'v3'

train_end_years = [2011, 2012, 2013, 2014, 2015]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])
  
  # get word2vec features
  def combine_with_word2vec_features(train_df, word2vec_features_filepath): #train_df is the training df, word2vec_features is the word2vec_features filepath(load data from /content/drive/My Drive/BT4222 Group/workspace/FILENAME)
    start_time = time.time()

    word2vec_features = pd.read_csv(word2vec_features_filepath,header=None) #read the file
    header_list = []
    for i in range(100):
        header_list.append("word2vec" + str(i+1))
    word2vec_features.columns = header_list #set column names
    word2vec_features["all"] = word2vec_features.values.tolist() #create column with all the values combined as list
    
    word2vec_features = shape_wv(word2vec_features, train_start, train_end, original_combined_df, industry_df)
    
    train_df = train_df.reset_index() #reset indexes for both train and word2vec feature df
    word2vec_features = word2vec_features.reset_index()

    train_df["word2vec_features"] = word2vec_features["all"] #put the combined list values into train df
    train_df = train_df.set_index('time') #set index as time again
    
    print(train_df.shape)

    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

    return train_df

  start, end = 2011, min(train_end_year, 2015)
  word2vec_features_path = "./Data/new_features_full_%d_%d_%dyr_word2vec.csv" % (start, end, end - start + 1)
  word2vec_model_path = "./Data/new_full_%d_%d_%dyr_word2vec.wv" % (start, end, end - start + 1)
  train_data = combine_with_word2vec_features(train_data, word2vec_features_path)
  print("imhere")

  word2vec_df = pd.DataFrame(train_data['word2vec_features'].tolist())
  word2vec_df.index = train_data.index
  word2vec_df.columns = ['word2vec_%d' % i for i in range(word2vec_df.shape[1])]

  train_data = pd.concat([train_data.iloc[:, :-1], word2vec_df], axis=1)

  # get test data word vectors
  stop_words = stopwords.words('english')

  test_data['headline'] = test_data['headline'].fillna("")
  test_data_words = []

  for headline in test_data['headline']:
    words = [word.lower() for word in word_tokenize(headline) if word.isalpha()]
    words = [word for word in words if not word in stop_words]
    test_data_words.append(words)


  # get features
  def get_features_from_word2vec(word2vec, headline_list, feature_size):
    start_time = time.time()

    features = [] # Each feature is a 1x100 vector
    word_not_in_vocabulary = []

    for words in headline_list:
        feature = np.zeros(feature_size)
        for word in words:
            try:
              feature += word2vec.wv.get_vector(word)
            except:
              continue
              #word_not_in_vocabulary.append(word)
        features.append((feature / float(len(words))).tolist())

    no_nans_feature_list = list(filter(lambda x: np.isnan(x).any() == False, features))
    transposed = zip(*no_nans_feature_list)
    avg = lambda items: float(sum(items))/ len(items)
    average_vector = list(map(avg, transposed)) #this average vector will be used to replace all nan vectors

    features = np.array(list(map(lambda x: x if np.isnan(x).any() == False else average_vector, features)), dtype='float16').tolist()

    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    return features

  word2vec_model = Word2Vec.load(word2vec_model_path)
  test_word2vec_features = get_features_from_word2vec(word2vec_model, test_data_words, 100)
#   word2vec_df_test = pd.DataFrame(test_word2vec_features)
#   word2vec_df_test.columns = ['word2vec_%d' % i for i in range(word2vec_df_test.shape[1])]
#   word2vec_df_test.index = test_data.index
#   test_data = pd.concat((test_data, word2vec_df_test), axis=1)

  word2vec_df_test = pd.DataFrame(test_word2vec_features)
  word2vec_df_test.columns = ['word2vec_%d' % i for i in range(word2vec_df_test.shape[1])]
  word2vec_df_test.index = test_data.index
  test_data = pd.concat((test_data, word2vec_df_test), axis=1)

  # get word2vec feature names
  word2vec_features_list = []
  for col in train_data.columns:
    if 'word2vec' in col:
      word2vec_features_list.append(col)
    
  #### insert wrapper code ####  
    
  
  # reshape data
  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values

      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

(413659, 50)
word2vec: 413659
(398990, 101)
00:00:13
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: RuntimeWarning: invalid value encountered in true_divide


00:00:12
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 101s 279us/step - loss: 0.6692 - acc: 0.5867 - val_loss: 0.7101 - val_acc: 0.5335
current sigma: 0.266762
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 110s 304us/step - loss: 0.6498 - acc: 0.6148 - val_loss: 0.7174 - val_acc: 0.5301
current sigma: 0.272214
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 92s 254us/step - loss: 0.6349 - acc: 0.6336 - val_loss: 0.7340 - val_acc: 0.5213
current sigma: 0.234246
Year 2011: sigma=0.234246
[[44842 43274]
 [36107 41593]]
(836368, 50)
word2vec: 836368
(807295, 101)
00:00:27
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: RuntimeWarning: invalid value encountered in true_divide


00:00:13
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 159s 207us/step - loss: 0.6701 - acc: 0.5891 - val_loss: 0.6825 - val_acc: 0.5697
current sigma: 0.847021
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 171s 224us/step - loss: 0.6547 - acc: 0.6109 - val_loss: 0.6892 - val_acc: 0.5669
current sigma: 0.830408
Year 2012: sigma=0.830408
[[41834 42594]
 [30306 53602]]
(1260731, 50)
word2vec: 1260731
(1216559, 101)
00:00:51
imhere


...

00:00:13
Train on 1170288 samples, validate on 160020 samples
Epoch 1/1
1170288/1170288 [==============================] - 239s 204us/step - loss: 0.6706 - acc: 0.5856 - val_loss: 0.7017 - val_acc: 0.5442
current sigma: 0.419029
Train on 1170288 samples, validate on 160020 samples
Epoch 1/1
1170288/1170288 [==============================] - 243s 208us/step - loss: 0.6576 - acc: 0.6053 - val_loss: 0.7121 - val_acc: 0.5441
current sigma: 0.409676
Year 2013: sigma=0.409676
[[41092 40322]
 [32625 45981]]
(1662485, 50)
word2vec: 1662485
(1605525, 101)
00:01:16
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: RuntimeWarning: invalid value encountered in true_divide


00:00:13
Train on 1555344 samples, validate on 165816 samples
Epoch 1/1
1555344/1555344 [==============================] - 314s 202us/step - loss: 0.6693 - acc: 0.5874 - val_loss: 0.7165 - val_acc: 0.5203
current sigma: 0.064587
Train on 1555344 samples, validate on 165816 samples
Epoch 1/1
1555344/1555344 [==============================] - 321s 206us/step - loss: 0.6565 - acc: 0.6065 - val_loss: 0.7273 - val_acc: 0.5106
current sigma: -0.023169
Year 2014: sigma=-0.023169
[[40949 39742]
 [41415 43710]]
(2095742, 50)
word2vec: 2095742
(2024360, 101)
00:02:03
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: RuntimeWarning: invalid value encountered in true_divide


00:00:14
Train on 1969884 samples, validate on 180432 samples
Epoch 1/1
1969884/1969884 [==============================] - 398s 202us/step - loss: 0.6726 - acc: 0.5811 - val_loss: 0.6875 - val_acc: 0.5485
current sigma: 0.501853
Train on 1969884 samples, validate on 180432 samples
Epoch 1/1
1969884/1969884 [==============================] - 407s 207us/step - loss: 0.6611 - acc: 0.5993 - val_loss: 0.6957 - val_acc: 0.5371
current sigma: 0.409455
Year 2015: sigma=0.409455
[[56806 29135]
 [54390 40101]]
Wall time: 1h 6min 29s


In [10]:
filename = "growing_final_output_from_results_dict_V3.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()

### V8

###### part 1

In [6]:
original_combined_df = pd.read_pickle('./Data/combined.p')

###############################################
### RESHAPING COMBINED_DF ACCORDING TO TIME ###
# month from 0 to 11
original_combined_df['month'] = original_combined_df['month'] - 1

# missing value imputation

group_mean_features = [
  'returnsClosePrevMktres1',
  'returnsOpenPrevMktres1',
  'returnsClosePrevMktres10',
  'returnsOpenPrevMktres10',
  'returnsOpenPrevMktres1_relative',
  'returnsOpenPrevMktres10_relative',
]

original_combined_df['headline'].fillna('', inplace=True)
original_combined_df['urgency_min'].fillna(5.0, inplace=True)

original_combined_df[group_mean_features] = original_combined_df.groupby("assetCode")[group_mean_features].transform(lambda x: x.fillna(x.mean()))

original_combined_df = original_combined_df[~original_combined_df['returnsClosePrevMktres10'].isnull()]

original_combined_df.fillna(0, inplace=True) ### <--- up to this point to get original df in same state

In [7]:
industry_df = pd.read_csv('./Data/IndustryData.csv')

In [8]:
def shape_wv(word2vec_features, train_start, train_end, combined_df, industry_df):

  combined_df = combined_df.set_index('time')[train_start: train_end]
  print(combined_df.shape)
  ### RESHAPING COMBINED_DF ACCORDING TO TIME ###
  ###############################################

  assetCode = combined_df['assetCode'].values

  print("word2vec:", len(word2vec_features))

  word2vec_features['assetCode'] = assetCode
  
  word2vec_features = pd.merge(word2vec_features, industry_df, how='inner', on='assetCode')
  word2vec_features = word2vec_features[pd.notnull(word2vec_features['Industry'])]
  word2vec_features = word2vec_features.drop(['Industry'], axis=1)
  
  return word2vec_features

In [9]:
%%time

v_str = 'v3'

file_counter = 1

train_end_years = [2011, 2012, 2013, 2014, 2015]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
  # a trick to get testing stats
  if train_end_year == 2016:
    train_end_year = 2015
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
    test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
    train_end_year = 2016
  else:
    train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('%d-12-31' % train_end_year)
    test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = combined_data.set_index('time')[train_start: train_end]
  test_data = combined_data.set_index('time')[test_start: test_end]

  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
  test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])
  
  # get word2vec features
  def combine_with_word2vec_features(train_df, word2vec_features_filepath): #train_df is the training df, word2vec_features is the word2vec_features filepath(load data from /content/drive/My Drive/BT4222 Group/workspace/FILENAME)
    start_time = time.time()

    word2vec_features = pd.read_csv(word2vec_features_filepath,header=None) #read the file
    header_list = []
    for i in range(100):
        header_list.append("word2vec" + str(i+1))
    word2vec_features.columns = header_list #set column names
    word2vec_features["all"] = word2vec_features.values.tolist() #create column with all the values combined as list
    
    word2vec_features = shape_wv(word2vec_features, train_start, train_end, original_combined_df, industry_df)
    
    train_df = train_df.reset_index() #reset indexes for both train and word2vec feature df
    word2vec_features = word2vec_features.reset_index()

    train_df["word2vec_features"] = word2vec_features["all"] #put the combined list values into train df
    train_df = train_df.set_index('time') #set index as time again
    
    print(train_df.shape)

    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

    return train_df

  start, end = 2011, min(train_end_year, 2015)
  word2vec_features_path = "./Data/new_features_full_%d_%d_%dyr_word2vec.csv" % (start, end, end - start + 1)
  word2vec_model_path = "./Data/new_full_%d_%d_%dyr_word2vec.wv" % (start, end, end - start + 1)
  train_data = combine_with_word2vec_features(train_data, word2vec_features_path)
  print("imhere")

  word2vec_df = pd.DataFrame(train_data['word2vec_features'].tolist())
  word2vec_df.index = train_data.index
  word2vec_df.columns = ['word2vec_%d' % i for i in range(word2vec_df.shape[1])]

  train_data = pd.concat([train_data.iloc[:, :-1], word2vec_df], axis=1)

  # get test data word vectors
  stop_words = stopwords.words('english')

  test_data['headline'] = test_data['headline'].fillna("")
  test_data_words = []

  for headline in test_data['headline']:
    words = [word.lower() for word in word_tokenize(headline) if word.isalpha()]
    words = [word for word in words if not word in stop_words]
    test_data_words.append(words)


  # get features
  def get_features_from_word2vec(word2vec, headline_list, feature_size):
    start_time = time.time()

    features = [] # Each feature is a 1x100 vector
    word_not_in_vocabulary = []

    for words in headline_list:
        feature = np.zeros(feature_size)
        for word in words:
            try:
              feature += word2vec.wv.get_vector(word)
            except:
              continue
              #word_not_in_vocabulary.append(word)
        features.append((feature / float(len(words))).tolist())

    no_nans_feature_list = list(filter(lambda x: np.isnan(x).any() == False, features))
    transposed = zip(*no_nans_feature_list)
    avg = lambda items: float(sum(items))/ len(items)
    average_vector = list(map(avg, transposed)) #this average vector will be used to replace all nan vectors

    features = np.array(list(map(lambda x: x if np.isnan(x).any() == False else average_vector, features)), dtype='float16').tolist()

    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    return features

  word2vec_model = Word2Vec.load(word2vec_model_path)
  test_word2vec_features = get_features_from_word2vec(word2vec_model, test_data_words, 100)
#   word2vec_df_test = pd.DataFrame(test_word2vec_features)
#   word2vec_df_test.columns = ['word2vec_%d' % i for i in range(word2vec_df_test.shape[1])]
#   word2vec_df_test.index = test_data.index
#   test_data = pd.concat((test_data, word2vec_df_test), axis=1)

  word2vec_df_test = pd.DataFrame(test_word2vec_features)
  word2vec_df_test.columns = ['word2vec_%d' % i for i in range(word2vec_df_test.shape[1])]
  word2vec_df_test.index = test_data.index
  test_data = pd.concat((test_data, word2vec_df_test), axis=1)

  # get word2vec feature names
  word2vec_features_list = []
  for col in train_data.columns:
    if 'word2vec' in col:
      word2vec_features_list.append(col)
        
  #### insert wrapper code ####  
  
  # df has multiple columns of wv, has assetCode
  def mean_wv_by_assetCode(df, wv_names):

    # group according to asset code
    grouped = df.groupby('assetCode')

    # then find the mean of the word vectors # this forms a dataframe with all the mean_word_vectors
    mean_wv_df = grouped[wv_names].transform('mean')

    # concatenate the dataframe with the asset code and industry so that the mean are identifiable
    # mean_wv_df = pd.concat([df[['assetCode', 'Industry']], mean_wv_df], axis = 1)
    mean_wv_df['assetCode'] = df['assetCode'].tolist()

    # remove duplicated rows, there would be many
    mean_wv_df = mean_wv_df.drop_duplicates()
    
    return mean_wv_df


  def best_k_clustering(X):
  
    #######
    # SVD #
    #######

    svd_transformer = TruncatedSVD(n_components = 8)
    X_embedded = svd_transformer.fit_transform(X)

    # %%time

    models = {}

    cluster_range = range(2, 21)


    for k in cluster_range:
        kmeans_model = KMeans(n_clusters=k)
        kmeans_model.fit_transform(X_embedded)
        models[k] = kmeans_model

    # %%time

    scores_list = []

    for k in cluster_range:
        model = models[k]
        score = silhouette_score(X_embedded, model.labels_)
        scores_list.append(score)

    best_model = models[scores_list.index(max(scores_list)) + 2]
    cluster_labels = best_model.labels_

    return cluster_labels, (best_model, svd_transformer)


  def generate_columns_features(df, columns):

    grouped = df.groupby(columns)

    score = lambda x: (x - x.mean()) / x.std()
    standard_df = grouped.transform(score)
    standard_df = standard_df.add_suffix('_stan')

    df = pd.concat([df, standard_df], axis = 1)
    return df


  def get_v3_train_test(train_word2vec, test_word2vec, wv_names):
    """
      train_word2vec and test_word2vec contain original data combined with added
      word2vec data for train and extracted word2vec data for test (validation)
    """
    global mean_wv_df, best_model, train_cluster_neut, test_cluster_neut
    mean_wv_df = mean_wv_by_assetCode(train_word2vec, wv_names)
    _, (best_model, svd_transformer) = best_k_clustering(mean_wv_df[wv_names])
    train_word2vec['Cluster'] = best_model.predict(svd_transformer.transform(train_data[wv_names]))
    test_word2vec['Cluster'] = best_model.predict(svd_transformer.transform(test_data[wv_names]))

    columns = ['time', 'Cluster']
    train_cluster_neut = generate_columns_features(train_word2vec, columns)
    test_cluster_neut = generate_columns_features(test_word2vec, columns)

    return train_cluster_neut, test_cluster_neut
  
  train_cluster_standardised, test_cluster_standardised = get_v3_train_test(train_data, test_data, word2vec_features_list)
  
  train_cluster_standardised_filename = 'train_cluster_standardised_' + str(file_counter) + ".p"
  train_fileobj = open(train_cluster_standardised_filename, "wb")
  pickle.dump(train_cluster_standardised, train_fileobj)
  train_fileobj.close()
  
  test_cluster_standardised_filename = 'test_cluster_standardised_' + str(file_counter) + ".p"
  test_fileobj = open(test_cluster_standardised_filename, "wb")
  pickle.dump(test_cluster_standardised, test_fileobj)
  test_fileobj.close()
  
  file_counter += 1
  print("file_counter:", file_counter)




(413659, 50)
word2vec: 413659
(398990, 101)
00:00:13
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide


00:00:13
file_counter: 2
(836368, 50)
word2vec: 836368
(807295, 101)
00:00:26
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide


00:00:12
file_counter: 3
(1260731, 50)
word2vec: 1260731
(1216559, 101)
00:00:41
imhere


...

00:00:13
file_counter: 4
(1662485, 50)
word2vec: 1662485
(1605525, 101)
00:01:00
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide


00:00:13
file_counter: 5
(2095742, 50)
word2vec: 2095742
(2024360, 101)
00:01:35
imhere


C:\Users\Wei Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:150: RuntimeWarning: invalid value encountered in true_divide


00:00:14
file_counter: 6
Wall time: 43min 47s


In [6]:
%%time

# remark for Wei Han: add sentiments and other industry-neutralised features from v5

v_str = 'v8'

header_list = []
for i in range(100):
    header_list.append("word2vec_" + str(i))

train_end_years = [1, 2, 3, 4, 5]

lstm_features_list = [
    'returnsClosePrevRaw1',
    'returnsOpenPrevRaw1',
    'returnsClosePrevMktres1',
    'returnsOpenPrevMktres1',
    'returnsClosePrevRaw10',
    'returnsOpenPrevRaw10',
    'returnsClosePrevMktres10',
    'returnsOpenPrevMktres10',
    'open_close',
    'oc_average',
    'turnover',
    'open_close_relative',
    'turnover_relative',
    'volume_relative',
    'returnsOpenPrevMktres1_relative',
    'returnsOpenPrevMktres10_relative',
    'returnsClosePrevRaw1_stan',
    'returnsOpenPrevRaw1_stan',
    'returnsClosePrevMktres1_stan',
    'returnsOpenPrevMktres1_stan',
    'returnsClosePrevRaw10_stan',
    'returnsOpenPrevRaw10_stan',
    'returnsClosePrevMktres10_stan',
    'returnsOpenPrevMktres10_stan',
    'open_close_stan',
    'oc_average_stan',
    'turnover_stan',
    'open_close_relative_stan',
    'turnover_relative_stan',
    'volume_relative_stan',
    'returnsOpenPrevMktres1_relative_stan',
    'returnsOpenPrevMktres10_relative_stan',
]


additional_features_list = [
    'urgency_min',
    'urgency_count',
    'takeSequence_min',
    'takeSequence_max',
    'bodySize_mean',
    'wordCount_mean',
    'sentenceCount_mean',
    'companyCount_mean',
    'marketCommentary_mean',
    'relevance_mean',
    'sentimentNegative_mean',
    'sentimentNeutral_mean',
    'sentimentPositive_mean',
    'sentimentWordCount_mean',
    'sentimentNegative_mean_stan',
    'sentimentNeutral_mean_stan',
    'sentimentPositive_mean_stan',
    'sentimentWordCount_mean_stan',
    'noveltyCount12H_mean',
    'noveltyCount24H_mean',
    'noveltyCount3D_mean',
    'noveltyCount5D_mean',
    'noveltyCount7D_mean',
    'volumeCounts12H_mean',
    'volumeCounts24H_mean',
    'volumeCounts3D_mean',
    'volumeCounts5D_mean',
    'volumeCounts7D_mean',
]

label_encoded_features = [
    'dayofweek',
    'month',
]

continuous_features_list = lstm_features_list + additional_features_list
features_list = lstm_features_list + additional_features_list + label_encoded_features

for train_end_year in train_end_years:
#   # a trick to get testing stats
#   if train_end_year == 2016:
#     train_end_year = 2015
#     train_start, train_end = pd.to_datetime('2011-01-01'), pd.to_datetime('2016-06-30')
#     test_start, test_end = pd.to_datetime('%d-07-01' % (train_end_year + 1)), pd.to_datetime('%d-12-31' % (train_end_year + 1))
#     train_end_year = 2016
#   else:
#     train_start, train_end = pd.to_datetime('%d-01-01' % train_end_year), pd.to_datetime('%d-12-31' % train_end_year)
#     test_start, test_end = pd.to_datetime('%d-01-01' % (train_end_year + 1)), pd.to_datetime('%d-06-30' % (train_end_year + 1))
  
  
  # divide into train and test periods

  train_data = pd.read_pickle("./Data/train_cluster_standardised_%d.p" % train_end_year)
  test_data = pd.read_pickle("./Data/test_cluster_standardised_%d.p" % train_end_year)
    
  def remove_duplicates_from_df(df):
    header_list = []
    for i in range(100):
        header_list.append("word2vec_" + str(i))
        header_list.append("word2vec_" + str(i) + "_stan")

    df.drop(header_list, axis=1, inplace=True) #remove word2vec_%d_stan columns
    
    column_names = df.columns.tolist()
    dup_dic = {}
    for i in range(len(column_names)):
        curr_pos = i
        if column_names[i] not in dup_dic:
            dup_dic[column_names[i]] = [1, curr_pos]
        elif column_names[i] in dup_dic:
            initial_pos = dup_dic[column_names[i]][1]
            dup_dic[column_names[i]][0] + 1
            dup_dic[column_names[i]] = [dup_dic[column_names[i]][0] + 1, initial_pos, curr_pos]

    columns_to_drop_index = []
    for k, v in dup_dic.items():
        if v[0] == 2:
            columns_to_drop_index.append(v[1])
    
    df_column_range = list(range(df.shape[1]))
    unique_columns_index_list = [index for index in df_column_range if index not in columns_to_drop_index]
    df = df.iloc[:, unique_columns_index_list] #return all unique columns
    
    return df
    
  train_data = remove_duplicates_from_df(train_data)
  test_data = remove_duplicates_from_df(test_data)
    
  print("Train_data shape:", train_data.shape)
  print("Test_data shape:", test_data.shape)
    
#   from sklearn.preprocessing import StandardScaler
#   scaler = StandardScaler()
#   train_data[continuous_features_list] = scaler.fit_transform(train_data[continuous_features_list])
#   test_data[continuous_features_list] = scaler.transform(test_data[continuous_features_list])

  # reshape data
  word2vec_features_list = []

  def reshape_data(data, sequence_len=21, features_list=features_list):
    data_list, ann_list, embedding_list, word2vec_list, target_list, returns_list, date_list = [], [], [], [], [], [], []

    for assetCode, asset_data in data.groupby('assetCode'):
      # binarize the target variable
      returns = asset_data['returnsOpenNextMktres10']
      target = (asset_data['returnsOpenNextMktres10'] > 0).astype(int)
      dates = asset_data.index

      # arrays
      lstm_data = asset_data[lstm_features_list].values
      ann_data = asset_data[additional_features_list].values
      embedding_data = asset_data[label_encoded_features].values
      word2vec_data = asset_data[word2vec_features_list].values
    
      col_mean = np.nanmean(lstm_data, axis = 0) 
      inds = np.where(np.isnan(lstm_data)) 
      lstm_data[inds] = np.take(col_mean, inds[1])

      reshaped_data = np.repeat(lstm_data[:, np.newaxis, :], sequence_len, axis=1)
      for i in range(reshaped_data.shape[1]):
        reshaped_data[:, i, :] = np.roll(reshaped_data[:, i, :], i, axis=0)

      # discard the top rows and reverse the order of the shifted sequence to reflect the correct order
      reshaped_data = reshaped_data[sequence_len-1:, ::-1, :]
      ann_data = ann_data[sequence_len-1:, :]
      embedding_data = embedding_data[sequence_len-1:, :]
      word2vec_data = word2vec_data[sequence_len-1:, :]
      target = target.values[sequence_len-1:]
      returns = returns.values[sequence_len-1:]
      dates = dates[sequence_len-1:]

      data_list.append(reshaped_data)
      ann_list.append(ann_data)
      embedding_list.append(embedding_data)
      word2vec_list.append(word2vec_data)
      target_list.append(target)
      returns_list.append(returns)
      date_list.append(dates)

    reshaped_data, ann_data, embedding_data, word2vec_data, target, returns, dates = np.concatenate(data_list), np.concatenate(ann_list), np.concatenate(embedding_list), np.concatenate(word2vec_list), np.concatenate(target_list), np.concatenate(returns_list), np.concatenate(date_list)

    return (reshaped_data, ann_data, embedding_data, word2vec_data), target, returns, dates

  x_train_tuple, y_train, train_returns, train_dates = reshape_data(train_data)
  x_test_tuple, y_test, test_returns, test_dates = reshape_data(test_data)

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_tuple
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_tuple
  
  # create model
  from keras.layers import Input, Embedding, LSTM, Dense
  from keras.models import Model

  lstm_params = {
      'batch_size': 252,
      'epochs': 5,
  }

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  # tweak to match with batch_size
  train_until = train_len - train_len % lstm_params['batch_size']
  test_until = test_len - test_len % lstm_params['batch_size']

  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec = x_train_lstm[:train_until], x_train_ann[:train_until], x_train_embedding[:train_until], x_train_word2vec[:train_until]
  y_train = y_train[:train_until]
  train_returns = train_returns[:train_until]
  train_dates = train_dates[:train_until]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec = x_test_lstm[:test_until], x_test_ann[:test_until], x_test_embedding[:test_until], x_test_word2vec[:test_until]
  y_test = y_test[:test_until]
  test_returns = test_returns[:test_until]
  test_dates = test_dates[:test_until]

  train_len = x_train_lstm.shape[0]
  test_len = x_test_lstm.shape[0]

  np.random.seed(1)
  train_shuffle = np.random.permutation(range(train_len))
  test_shuffle = np.random.permutation(range(test_len))
  x_train_lstm, x_train_ann, x_train_embedding, x_train_word2vec, y_train, train_returns, train_dates = x_train_lstm[train_shuffle], x_train_ann[train_shuffle], x_train_embedding[train_shuffle], x_train_word2vec[train_shuffle], y_train[train_shuffle], train_returns[train_shuffle], train_dates[train_shuffle]
  x_test_lstm, x_test_ann, x_test_embedding, x_test_word2vec, y_test, test_returns, test_dates = x_test_lstm[test_shuffle], x_test_ann[test_shuffle], x_test_embedding[test_shuffle], x_test_word2vec[test_shuffle], y_test[test_shuffle], test_returns[test_shuffle], test_dates[test_shuffle]

  def create_model():
    sequence_len = x_train_lstm.shape[1]

    lstm_input = Input(shape=(sequence_len, len(lstm_features_list)), name='lstm_input')
    lstm_out = LSTM(21)(lstm_input)

    additional_input = Input(shape=(len(additional_features_list),), name='additional_input')
  #   word2vec_input = Input(shape=(len(word2vec_features_list),), name='word2vec_input')

  #   concat_layers = [lstm_out, additional_input, word2vec_input]
    concat_layers = [lstm_out, additional_input]

  #   input_layers = [lstm_input, additional_input, word2vec_input]
    input_layers = [lstm_input, additional_input]

    # embeddings
    for i in range(x_train_embedding.shape[1]):
      vocab_size = x_train_embedding[:, i].max() + 1
      y = Input(shape=(1,), dtype='int32', name='embedding_input_%d' % i)
      input_layers.append(y)
      y = Embedding(input_dim=vocab_size, output_dim=21)(y)
      concat_layers.append(Flatten()(y))

    x = keras.layers.concatenate(concat_layers)

    x = Dense(64, activation='relu')(x)
    x = Dropout(.2)(x)

    main_output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=input_layers, outputs=[main_output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
  
  def get_sigma(scaled_scores, test_returns, test_dates):
    '''
    scaled_scores: numpy array
    test_returns: numpy array
    test_dates: numpy array
    '''

    grouped_dict = pd.Series(scaled_scores).index.groupby(test_dates)

    day_scores = []
    for ts in grouped_dict:
      grouped_index = grouped_dict[ts]
      day_score = np.dot(test_returns[grouped_index], scaled_scores[grouped_index])
      day_scores.append(day_score)

    return np.mean(day_scores) / np.std(day_scores, ddof=1)


  prev_sigma = None
  combined_model = create_model()

  # train for at most 500 epochs
  for i in range(500):
    # x_train_list = [x_train_lstm, x_train_ann] + [x_train_word2vec] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    x_train_list = [x_train_lstm, x_train_ann] + [x_train_embedding[:, i] for i in range(x_train_embedding.shape[1])]
    # x_test_list = [x_test_lstm, x_test_ann] + [x_test_word2vec] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]
    x_test_list = [x_test_lstm, x_test_ann] + [x_test_embedding[:, i] for i in range(x_test_embedding.shape[1])]

    combined_model.fit(x_train_list,
                   y_train,
                   batch_size=lstm_params['batch_size'],
                   epochs=1,
                   verbose=1,
                   validation_data=(x_test_list, y_test),
                   shuffle=False)

    scaled_scores = combined_model.predict(x_test_list, batch_size=lstm_params['batch_size']).reshape(-1) * 2 - 1
    current_sigma = get_sigma(scaled_scores, test_returns, test_dates)
    
    print('current sigma: %f' % current_sigma)

    # early stopping if sigma score does not improve for 1 epoch
    if prev_sigma is not None and current_sigma < prev_sigma:
      break
    else:
      prev_sigma = current_sigma
  
  pred_classes = (scaled_scores > 0).astype(int)
  from sklearn.metrics import confusion_matrix
  conf_mat = confusion_matrix(y_test, pred_classes)
  
  print('Year %d: sigma=%f' % (train_end_year, current_sigma))
  print(conf_mat)
  
  if v_str not in results:
    results[v_str] = {}
  results[v_str][train_end_year] = {}
  results[v_str][train_end_year]['y_test'] = y_test
  results[v_str][train_end_year]['scaled_scores'] = scaled_scores
  results[v_str][train_end_year]['sigma_score'] = current_sigma
  results[v_str][train_end_year]['conf_mat'] = conf_mat

Train_data shape: (398990, 148)
Test_data shape: (201982, 148)
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 72s 200us/step - loss: 0.6933 - acc: 0.5005 - val_loss: 0.6931 - val_acc: 0.5314
current sigma: 0.365377
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 81s 224us/step - loss: 0.6932 - acc: 0.5008 - val_loss: 0.6931 - val_acc: 0.5314
current sigma: 0.365377
Train on 361116 samples, validate on 165816 samples
Epoch 1/1
361116/361116 [==============================] - 86s 239us/step - loss: 0.6932 - acc: 0.5008 - val_loss: 0.6931 - val_acc: 0.5314
current sigma: 0.365377
Year 1: sigma=0.365377
[[88116     0]
 [77700     0]]
Train_data shape: (807295, 148)
Test_data shape: (205057, 148)
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 169s 221us/step - loss: 0.6931 - acc: 0.5080 - val_loss: 0.69

764820/764820 [==============================] - 169s 221us/step - loss: 0.6930 - acc: 0.5083 - val_loss: 0.6933 - val_acc: 0.4985
current sigma: -0.160585
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 167s 218us/step - loss: 0.6930 - acc: 0.5083 - val_loss: 0.6933 - val_acc: 0.4985
current sigma: -0.160585
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 166s 217us/step - loss: 0.6930 - acc: 0.5083 - val_loss: 0.6933 - val_acc: 0.4985
current sigma: -0.160585
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 171s 224us/step - loss: 0.6930 - acc: 0.5083 - val_loss: 0.6933 - val_acc: 0.4985
current sigma: -0.160585
Train on 764820 samples, validate on 168336 samples
Epoch 1/1
764820/764820 [==============================] - 168s 219us/step - loss: 0.6930 - acc: 0.5083 - val_loss: 0.6933 - val_acc: 0.49

KeyboardInterrupt: 

In [7]:
filename = "growing_final_output_from_results_dict_V8.p"
fileobj = open(filename, "wb")
pickle.dump(results, fileobj)
fileobj.close()